In [55]:
#importing relevant libraries
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score

In [2]:
#displaying the application data set as a dataframe
df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv")
pd.set_option('display.max_columns',50)
display(df.head())

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [3]:
df.describe()

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
count,1462.000000,1281.000000,1462.000000,1462.000000,1462.000000
mean,2.031464,59886.273224,2.976744,0.506108,0.619015
std,1.449717,15070.140389,1.681564,0.288465,0.485795
min,0.000000,13929.000000,0.000000,0.000000,0.000000
25%,1.000000,49698.000000,2.000000,0.262500,0.000000
50%,2.000000,60148.000000,3.000000,0.510000,1.000000
75%,3.000000,69639.000000,4.000000,0.750000,1.000000
max,9.000000,109899.000000,11.000000,1.000000,1.000000


## Data preparation

In [4]:
# How many columns in the dataset have missing values?
print(df.isnull().sum()[df.isnull().sum() > 0])

lead_source          128
industry             134
annual_income        181
employment_status    100
location              63
dtype: int64


In [5]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [6]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [24]:
#numerical_features
numerical_features = ["number_of_courses_viewed", "interaction_count","annual_income",  "lead_score"]

# Categorical features
categorical_features = ['industry', 'location', 'lead_source', 'employment_status']

In [37]:
# Replace missing values
df[categorical_features] = df[categorical_features].fillna('NA')
df[numerical_features] = df[numerical_features].fillna(0.0)
features = numerical_features + categorical_features

In [49]:
# Separate features and target
y_train = df_train['converted'].values
y_val = df_val['converted'].values

NameError: name 'df_train' is not defined

In [50]:
# First split: train (60%) and temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=1)

In [51]:
# Second split: val (20%) and test (20%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

In [52]:
auc_scores = {}

for feature in numerical_features:
    score = roc_auc_score(y, df[feature])
    if score < 0.5:
        score = roc_auc_score(y, df[feature])
    auc_scores[feature] = score

In [53]:
# Finding the feature with highest AUC
best_feature = max(auc_scores, key=auc_scores.get)

print("AUC scores:", auc_scores)
print("Highest AUC feature:", best_feature)

AUC scores: {'number_of_courses_viewed': 0.7608270430582144, 'interaction_count': 0.7223206403681919, 'annual_income': 0.5426406260848865, 'lead_score': 0.6148050428003213}
Highest AUC feature: number_of_courses_viewed


In [54]:
# One-hot encode
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

# Train logistic regression
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
model.fit(X_train, y_train)

# Predict probabilities
y_pred = model.predict_proba(X_val)[:, 1]

# Compute AUC
auc = roc_auc_score(y_val, y_pred)
print("Validation AUC:", round(auc, 3))


ValueError: Found input variables with inconsistent numbers of samples: [1462, 877]

In [56]:
thresholds = np.arange(0.0, 1.01, 0.01)
precisions = []
recalls = []

for t in thresholds:
    y_pred_label = (y_pred >= t).astype(int)
    precisions.append(precision_score(y_val, y_pred_label))
    recalls.append(recall_score(y_val, y_pred_label))

NameError: name 'y_pred' is not defined